In [18]:
%pylab inline
import pandas as pd
from sys import platform
import matplotlib.pyplot as plt
import numpy as np
from scipy import stats
from geopy.geocoders import Nominatim
import folium

if platform == "win32":
    path = 'C:/Users/olive/GitHub/f1-analytics/'
elif platform == "darwin":
    path = '~/Documents/GitHub/f1-analytics/'
    

%pylab is deprecated, use %matplotlib inline and import the required libraries.
Populating the interactive namespace from numpy and matplotlib


In [19]:
### Load race and mereged DataFrames

data = pd.read_csv(path + 'data/results.csv')
merged = pd.read_csv(path + 'data/merged.csv')

In [20]:
incidents = data.query(
    'status != "Finished"'
).agg({
    'status': 'unique'
})

In [21]:
incidents

status    [Suspension, Electronics, Wheel, Fuel pressure...
dtype: object